In [1]:
import pandas as pd
import requests
import json
from dateutil import parser
import time

In [2]:
URL = ""
API_KEY = ""
ACCOUNT_ID_1 = ""
ACCOUNT_ID_2 = ""



In [4]:
session = requests.Session()


In [5]:
headers= {
    "Authorization" : f"bearer {API_KEY}",
    "Content_Type": "application/json"
}

## SEND HEADERS WITH EVRY REQUEST

In [6]:
session.headers.update(headers)

In [7]:
url_candles = f"{URL}/instruments/EUR_USD/candles"
url_instruments = f"{URL}/accounts/{ACCOUNT_ID_2}/instruments"


In [ ]:
## Get isnstruments

In [8]:
respons_2 = session.get(url_instruments, params= None)

In [9]:
respons_2.status_code

200

In [10]:
data =respons_2.json()

In [11]:
instruments_list =data['instruments']

In [12]:
len(instruments_list)

128

In [13]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'minimumGuaranteedStopLossDistance', 'guaranteedStopLossOrderExecutionPremium', 'guaranteedStopLossOrderLevelRestriction', 'tags', 'financing'])

In [14]:
instruments_list[0]

{'name': 'XAG_USD',
 'type': 'METAL',
 'displayName': 'Silver',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1',
 'maximumTrailingStopDistance': '1.00000',
 'minimumTrailingStopDistance': '0.00050',
 'maximumPositionSize': '0',
 'maximumOrderUnits': '500000',
 'marginRate': '0.1',
 'guaranteedStopLossOrderMode': 'ALLOWED',
 'minimumGuaranteedStopLossDistance': '0.2400',
 'guaranteedStopLossOrderExecutionPremium': '0.02999999999998',
 'guaranteedStopLossOrderLevelRestriction': {'volume': '25000',
  'priceRange': '0.25'},
 'tags': [{'type': 'ASSET_CLASS', 'name': 'COMMODITY'},
  {'type': 'KID_ASSET_CLASS', 'name': 'METAL'},
  {'type': 'BRAIN_ASSET_CLASS', 'name': 'METAL'}],
 'financing': {'longRate': '-0.0688',
  'shortRate': '0.0463',
  'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
   {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
   {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
   {'dayOfWeek': 'THURSDAY', 'daysCharged':

In [15]:
key_i =['name', 'type', 'displayName', 'pipLocation', 'displayPrecision','tradeUnitsPrecision', 'marginRate']

In [16]:
instruments_dict = {}
for instr in instruments_list:
    key =instr['name']
    instruments_dict[key]= {k:instr[k] for k in key_i}


In [17]:
instruments_dict

{'XAG_USD': {'name': 'XAG_USD',
  'type': 'METAL',
  'displayName': 'Silver',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.1'},
 'USB30Y_USD': {'name': 'USB30Y_USD',
  'type': 'CFD',
  'displayName': 'US T-Bond',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.2'},
 'USB05Y_USD': {'name': 'USB05Y_USD',
  'type': 'CFD',
  'displayName': 'US 5Y T-Note',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.2'},
 'NZD_JPY': {'name': 'NZD_JPY',
  'type': 'CURRENCY',
  'displayName': 'NZD/JPY',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'EUR_CZK': {'name': 'EUR_CZK',
  'type': 'CURRENCY',
  'displayName': 'EUR/CZK',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'XAU_AUD': {'name': 'XAU_AUD',
  'type': 'METAL',
  'displayName': 'Gold/AUD',
  'pipLocatio

In [ ]:
with open('../data/intsruments.json', 'w') as f:
    f.write(json.dumps(instruments_dict, indent =2))


In [18]:
instruments_dict['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.03333333333333'}

In [ ]:
# GET CANDLES

In [ ]:
params =  dict(
    granularity = "H4",
    count = 10,
    price = "MBA"    
)



In [ ]:
prices = ["bid","mid","ask"]
ohlc = ["o","h","l","c"]
for p in prices:
    for x in ohlc:
        key = f"{p}_{x}"
        

In [39]:
def fetch_candles(pair_name, count=10,granularity = "H1"):
    url = f"{URL}/instruments/{pair_name}/candles"
    
    params =  dict(
    granularity = granularity,
    count = count,
    price = "MBA" )
    
    respons= session.get(url, params = params)
    data = respons.json()
    if respons.status_code == 200:
        if "candles" not in data:
            data =[]
        else:
             data =data["candles"]

    
    return respons.status_code , data

def get_candles_df (data):    
    if len(data) == 0:        
        return df.DataFrame()
    else:
        final_data = []
        for candle in data:
            if candle["complete"] == False:
                continue
            else:
                new_dict = {}
                new_dict["time"] = parser.parse(candle["time"])
                new_dict["volume"] = candle["volume"]
                prices = ["bid","mid","ask"]
                ohlc = ["o","h","l","c"]
                for p in prices:
                    for x in ohlc:
                        key = f"{p}_{x}"
                        new_dict[key]=float(candle[p][x])
                
                final_data.append(new_dict)
        df = pd.DataFrame.from_dict(final_data)
        return df


def create_data_file(pair_name, count=10,granularity = "H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("FAILED", pair_name)
        return
    if len(data) == 0:
        print("NO CANDLES", pair_name)
        
    candles_df = get_candles_df(data)   
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"Saved to file: {pair_name} {granularity} {candles_df.shape[0]} {candles_df.time.min()} - {candles_df.time.max()}")
    

In [42]:
create_data_file("EUR_USD", count =100, granularity = "H1")

Saved to file: EUR_USD H1 100 2023-11-20 18:00:00+00:00 - 2023-11-24 21:00:00+00:00


In [19]:
our_curr = ["EUR", "USD","GBP","JPY","CHF", "NZD", "CAD","AUD"]
 

In [43]:
for c1 in our_curr:
    for c2 in our_curr:
        pair = f"{c1}_{c2}"
        if pair  in  instruments_dict:
            for g in ["H1", "H4"]:                
                create_data_file(pair, count = 4001, granularity= g)
                time.sleep(4)
       

Saved to file: EUR_USD H1 4001 2023-04-06 04:00:00+00:00 - 2023-11-24 21:00:00+00:00
Saved to file: EUR_USD H4 4001 2021-05-05 01:00:00+00:00 - 2023-11-24 18:00:00+00:00
Saved to file: EUR_GBP H1 4001 2023-04-06 04:00:00+00:00 - 2023-11-24 21:00:00+00:00
Saved to file: EUR_GBP H4 4001 2021-05-05 01:00:00+00:00 - 2023-11-24 18:00:00+00:00
Saved to file: EUR_JPY H1 4001 2023-04-06 04:00:00+00:00 - 2023-11-24 21:00:00+00:00
Saved to file: EUR_JPY H4 4001 2021-05-04 17:00:00+00:00 - 2023-11-24 18:00:00+00:00
Saved to file: EUR_CHF H1 4001 2023-04-06 04:00:00+00:00 - 2023-11-24 21:00:00+00:00
Saved to file: EUR_CHF H4 4001 2021-05-05 01:00:00+00:00 - 2023-11-24 18:00:00+00:00
Saved to file: EUR_NZD H1 4001 2023-04-06 03:00:00+00:00 - 2023-11-24 21:00:00+00:00
Saved to file: EUR_NZD H4 4001 2021-05-05 01:00:00+00:00 - 2023-11-24 18:00:00+00:00
Saved to file: EUR_CAD H1 4001 2023-04-06 04:00:00+00:00 - 2023-11-24 21:00:00+00:00
Saved to file: EUR_CAD H4 4001 2021-05-05 01:00:00+00:00 - 2023-1